In [1]:
import pymc3 as pm
import theano
import theano.tensor as tt

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('ggplot')

In [2]:
# df = pd.read_csv('weekly_dmd.csv')
df = pd.read_csv('weekly_dmd_new.csv')
df['date'] = pd.to_datetime(df.date).apply(lambda x: x.date())
df = df.set_index(df.date)
df.index.rename('date_idx', inplace=True)
df.head(2)

,zone_id,date,total_demand_new,total_demand_new_lag1,total_demand_new_lag2,total_demand_new_lag3,total_demand_new_lag-1,total_demand_new_lag-2,total_demand_new_lag-3,smoothed_total_demand,year,total_demand_new_lag4,t4w_total_demand,month,week,day,t4w_total_demand_lag2,szn_factor
date_idx,,,,,,,,,,,,,,,,,,
2017-07-10,1,2017-07-10,2156.459303,NaN,NaN,NaN,2110.494687,2194.632691,2186.039578,NaN,2017,NaN,NaN,7,28,10,NaN,NaN
2017-07-10,597,2017-07-10,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,2017,NaN,NaN,7,28,10,NaN,NaN


In [3]:
df.date.max()

datetime.date(2021, 7, 5)

In [4]:
covid_start_idx = date(2020,2,17)

df_prep = df.loc[(df.date < covid_start_idx)
                | (df.date >= covid_start_idx + timedelta(weeks=52))].sort_values('date')

df_prep['y_obs'] = df_prep.total_demand_new
# df_prep['y_obs'] = df_prep.smoothed_total_demand
df_prep['covid_start_m1'] = df_prep.date.apply(lambda x: 1.0 if x == covid_start_idx - timedelta(weeks=1) else 0)

In [13]:
def fit_hw_model(zdf, initial_lvl_upper):
    szn_len = 52

    with pm.Model() as hw_model:
    #     initial_level = pm.Normal(f'initial_level', mu=3000, sigma=400)
#         initial_level = pm.Uniform(f'initial_level', lower=0, upper=5000)
        initial_level = pm.Uniform(f'initial_level', lower=0, upper=initial_lvl_upper)
        smoothing_level = pm.Uniform('smoothing_level', lower=0, upper=1)
#         smoothing_level = pm.Normal('smoothing_level', mu=0.5, sigma=0.5)

    #     initial_trend = pm.Normal(f'initial_trend', mu=0, sigma=100)
#         initial_trend = pm.Uniform(f'initial_trend', lower=-1000, upper=1000)
        initial_trend = pm.Uniform(f'initial_trend', lower=-initial_lvl_upper/2, upper=initial_lvl_upper/2)
        smoothing_trend = pm.Uniform('smoothing_trend', lower=0, upper=1)
    #     smoothing_trend = pm.Normal('smoothing_trend', mu=0.5, sigma=0.5)

        if len(zdf) < szn_len*2.5:
#         if len(zdf) < szn_len*2:
#             initial_szns = pm.Deterministic('initial_szns', 1.0, dims=(szn_len))
            initial_szns = tt.as_tensor_variable([1.0 for i in range(52)])
            smoothing_season = 0.0
#             smoothing_season = pm.Deterministic('smoothing_season', 1.0)
        else:
            initial_szns = pm.Normal('initial_szns', mu=1.0, sigma=0.2, shape=(szn_len))
        #     initial_szns = pm.Uniform('initial_szns', lower=0.7, upper=1.3, shape=(szn_len))
        #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
        #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
        #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1-smoothing_level)
            smoothing_season = pm.TruncatedNormal('smoothing_season', mu=0.2, sigma=0.5, lower=0, upper=0.8)
        #     smoothing_season = pm.Normal('smoothing_season', mu=0.5, sigma=0.5)

        ys = tt.as_tensor_variable(zdf['y_obs'])
        is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
    #     covid_level_change = pm.Normal('covid_level_change', mu=15000, sigma=2000)
        covid_level_change = pm.Uniform('covid_level_change', lower=0, upper=5000)
#         covid_level_change = pm.Normal('covid_level_change', mu=2000, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=20000, sigma=2000)
        covid_level_change2 = pm.Uniform('covid_level_change2', lower=0, upper=5000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)

        def hw_component_wise(y, covid_start, prior_level, prior_trend, szn_m52):
            updated_szn = smoothing_season * y / (prior_level + prior_trend) + (1 - smoothing_season) * szn_m52


            next_level = smoothing_level * (y/szn_m52) + (1 - smoothing_level)*(prior_level + prior_trend) + covid_start*covid_level_change
            next_trend = smoothing_trend * (next_level - prior_level) + (1 - smoothing_trend)*prior_trend - covid_start*smoothing_trend*covid_level_change2


            return next_level, next_trend, updated_szn

        outputs, updates = theano.scan(hw_component_wise,
               sequences = [
                   ys[1:],
#                    ys[szn_len:],
                   is_covid_start[1:]
#                    is_covid_start[szn_len:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = initial_level, taps=None),
                   dict(initial = initial_trend, taps=None),
                   dict(initial = initial_szns, taps=[-szn_len])
               ], 
        #            non_sequences = 
               )


        levels = outputs[0]
        trends = outputs[1]
        seasons = outputs[2]


        levels_f = pm.math.concatenate([initial_level.reshape(1,1), levels[:-1]])
        trends_f = pm.math.concatenate([initial_trend.reshape(1,1), trends[:-1]])
        seasons_f = pm.math.concatenate([initial_szns, seasons[:-szn_len]])

        levels_and_trends = pm.math.stack([levels_f, trends_f])
        level_plus_trend = levels_and_trends.sum(axis=0)

        level_plus_trend_and_seasons = pm.math.stack([level_plus_trend, seasons_f])
        y_hats = level_plus_trend_and_seasons.prod(axis=0)

        sig = pm.HalfCauchy('sigma', beta=10)
#         y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[szn_len-1:-1, :]['y_obs_lag-1'])
        y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[:-1, :]['y_obs_lag-1'])
        
    map_estimate = pm.find_MAP(model=hw_model)
    
    return map_estimate, ys, is_covid_start

In [14]:
def predict(map_estimate, ys, is_covid_start, horizon):
    
    if 'initial_szns' in map_estimate:
        szn_estimates = tt.as_tensor_variable(map_estimate['initial_szns'])
        szn_smoother = map_estimate['smoothing_season']
    else:
        szn_estimates = tt.as_tensor_variable([1.0 for i in range(52)])
        szn_smoother = 0.0

    def hw_component_wise_final(y, covid_start, prior_level, prior_trend, szn_m52):
            updated_szn = szn_smoother * y / (prior_level + prior_trend) + (1 - szn_smoother) * szn_m52
#             updated_szn = map_estimate['smoothing_season'] * y / (prior_level + prior_trend) + (1 - map_estimate['smoothing_season']) * szn_m52


            next_level = map_estimate['smoothing_level'] * (y/szn_m52) + (1 - map_estimate['smoothing_level'])*(prior_level + prior_trend) + covid_start*map_estimate['covid_level_change']
            next_trend = map_estimate['smoothing_trend'] * (next_level - prior_level) + (1 - map_estimate['smoothing_trend'])*prior_trend - covid_start*map_estimate['smoothing_trend']*map_estimate['covid_level_change2']


            return next_level, next_trend, updated_szn

    outputs, updates = theano.scan(hw_component_wise_final,
               sequences = [
#                    ys[szn_len:],
                   ys[1:],
#                    is_covid_start[szn_len:]
                   is_covid_start[1:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = map_estimate['initial_level'], taps=None),
                   dict(initial = map_estimate['initial_trend'], taps=None),
                   dict(initial = szn_estimates, taps=[-szn_len])
#                    dict(initial = tt.as_tensor_variable(map_estimate['initial_szns']), taps=[-szn_len])
               ], 
        #            non_sequences = 
               )
    
    levels = outputs[0]
    trends = outputs[1]
    seasons = outputs[2]

    levels_f = pm.math.concatenate([map_estimate['initial_level'].reshape(1), levels[:-1]])
    trends_f = pm.math.concatenate([map_estimate['initial_trend'].reshape(1), trends[:-1]])
#     seasons_f = pm.math.concatenate([map_estimate['initial_szns'], seasons[:-szn_len]])
    seasons_f = pm.math.concatenate([szn_estimates, seasons[:-szn_len]])

    levels_and_trends = pm.math.stack([levels_f, trends_f])
    level_plus_trend = levels_and_trends.sum(axis=0)

    level_plus_trend_and_seasons = pm.math.stack([level_plus_trend, seasons_f])
    y_hats = level_plus_trend_and_seasons.prod(axis=0)
    
    corresponding_szn_idx = (horizon - szn_len) - 1
#     szn_estimate = seasons_f.get_test_value()[corresponding_szn_idx]
    szn_estimate = seasons.get_test_value()[corresponding_szn_idx]

#     pred = (levels_f.get_test_value()[-1] + horizon*trends_f.get_test_value()[-1])*szn_estimate
    pred = (levels.get_test_value()[-1] + horizon*trends.get_test_value()[-1])*szn_estimate
#     return pred, levels, trends, seasons, corresponding_szn_idx, szn_estimate, y_hats
    return pred, levels_f, trends_f, seasons_f, seasons, corresponding_szn_idx, szn_estimate, y_hats

In [9]:
latest_f_date = df_prep.date.max() + timedelta(weeks=1)
latest_f_date

datetime.date(2021, 7, 12)

In [10]:
zz = [18,  307,  438,  266,   90,  541,  272,  692,  475,  348,  273,
        178,  495,  694,  190,  707,  161,  521,  452,  489,  785,  942,
        972,  861,  973,  943,  975,  981,  974, 1002,  983, 1001,  839,
        714,    1,  783,  989,  787,   79,  945,  169,   37,  186,   76,
        620,  441,   32,  184,  477,  357,  530,  616,  167,  617,  437,
         38,  166, 1029,  428,  853,   43,   16,  982,  713,  988,  262,
        840,  144,  709,  181,  204,  440,  225,  245,  185,  104,  182,
        103,  439,   50,  946,  980,   27,  979,  183,  786,  131,   33,
        392,   87]

In [17]:
HORIZON = 2
szn_len = 52

In [18]:
### walk-forward validation

idx_outer = 1
pred_for_dates, zone_ids, preds, smoothing_levels, smoothing_trends, smoothing_seasons = [], [], [], [], [], []
for f_date in [latest_f_date]:
#     print(f'iter {idx_outer}/{len(test_dates2)}')
#     idx_outer += 1
    
    idx_inner = 1
    for z_id in zz:
        print(f'inner iter {idx_inner}')
        
        zdf = df_prep.loc[(df_prep.zone_id == z_id)
                            & (df_prep.date < f_date)
                            & (df_prep['y_obs'] > 0)
                           ]
    
        third_week = zdf.date.unique()[2]
        zdf = zdf.loc[zdf.date > third_week]
        
        
        pad_df = pd.DataFrame(columns = zdf.columns)
        pad_df.loc[0, 'zone_id'] = z_id
        zdf = pd.concat([pad_df, zdf])
        
        zdf['y_obs_lag-1'] = zdf.y_obs.shift(-1)
        initial_lvl_upper = zdf.head()['y_obs_lag-1'].max()
        
        map_estimate, ys, is_covid_start = fit_hw_model(zdf, initial_lvl_upper)
        pred, levels_f, trends_f, seasons_f, seasons, corresponding_szn_idx, szn_estimate, yhats = predict(map_estimate, ys, is_covid_start, HORIZON+1)
        
        pred_for_date = f_date + timedelta(weeks=HORIZON)
        
        pred_for_dates.append(pred_for_date)
        zone_ids.append(z_id)
        preds.append(pred)
        smoothing_levels.append(map_estimate['smoothing_level'])
        smoothing_trends.append(map_estimate['smoothing_trend'])
        if 'smoothing_season' in map_estimate:
            smoothing_seasons.append(map_estimate['smoothing_season'])
        else:
            smoothing_seasons.append(0.0)
        idx_inner += 1
        
res = pd.DataFrame({'pred_for_date': pred_for_dates,
                        'zone_id': zone_ids,
                        'pred': preds,
                        'smoothing_level': smoothing_levels,
                        'smoothing_trend': smoothing_trends,
                        'smoothing_season': smoothing_seasons})

res.pred_for_date = pd.to_datetime(res.pred_for_date)

inner iter 1


WARNING (theano.scan.basic): Output initial_level (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output initial_trend (index 1) has a initial state but taps is explicitly set to None 


WARNING (theano.scan.basic): Output None (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output None (index 1) has a initial state but taps is explicitly set to None 



inner iter 2


WARNING (theano.scan.basic): Output initial_level (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output initial_trend (index 1) has a initial state but taps is explicitly set to None 


...

WARNING (theano.scan.basic): Output initial_level (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output initial_trend (index 1) has a initial state but taps is explicitly set to None 


WARNING (theano.scan.basic): Output None (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output None (index 1) has a initial state but taps is explicitly set to None 



inner iter 90


WARNING (theano.scan.basic): Output initial_level (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output initial_trend (index 1) has a initial state but taps is explicitly set to None 


WARNING (theano.scan.basic): Output None (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output None (index 1) has a initial state but taps is explicitly set to None 


In [ ]:
# [] produce demand forecasts for f_date
    # [] translate to DoW using existing method for DoW distribution
# [] produce supply forecasts for f_date
    # [] pull historical melt rates and just use DES for this
        # [] can do some quick backtest here to see if we want to use the covid-handling version or out-of-box
    # [] use existing in-funnel estimate
    
# [] combine with ratios
    # [] compare results to what they would be with perfect forecasts

# [] finish implementing hierarchical version while existing forecasts are running
    # [] this will help get around the issues currently facing with not having enough data to get seasonal estimates in many places
        
# [] maybe should consider taking median (not mean) when grouping by week, since the DoW adjustments are generated relative to period medians
# [] could also calculate the relative DoW factors in here, since it's not much code, so that it's being done on the adjusted demand data (to account for holiday effects)

# [] look at correlation w/ recent and solve week LD, avail, TTA for old and new
# [] look at zones where old didnt have gap and LD was high (or other metrics low)
    # [] if we didn't have a gap, would we have a gap even with perfect forecast?
# [] vice versa for above

# [] shouldnt we actually be able to get accuracy improvement impacts from backtesting too?
    # [] ie, we can see what sage would have done w/ perfect forecasts

In [ ]:
### translate to DoW using existing method for DoW distribution
q = """
select
    zone_id,
    forecast_date,
    pred_for_date,
    parse_json(data)['daily_total_visits_relative_to_period_median'] as demand_dow_dist,
    parse_json(data)['daily_melt_for_horizon_relative_to_period_median'] as supply_dow_dist
from ml.zone_day_supply_gaps g
where forecast_date = {f_date}
"""

daily_dist = lore.io.snowflake.dataframe(q, f_date=latest_f_date)